# NFL Capstone:Modeling

### Starting Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import os
import sys

## Import datasets

In [2]:
year = pd.read_csv('../data/teamstarterdraft.csv')
yearAV = pd.read_csv('../data/teamstarterdraftAV.csv')
week = pd.read_csv('../data/weekstarterdraft.csv')
weekAV = pd.read_csv('../data/weekstarterdraftAV.csv')
yearnocoach = year.drop(columns=['coach', 'offcoor', 'defcoor', 'offscheme', 'defalign'])
yearnocoachAV = yearAV.drop(columns=['coach', 'offcoor', 'defcoor', 'offscheme', 'defalign'])

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

#this is what I changed from cell above
dfdict = {'year':year, 'yearAV':yearAV, 'week':week,'weekAV':weekAV, 'yearnocoach': yearnocoach, 'yearnocoachAV':yearnocoachAV}


dfresults= pd.DataFrame()
for key, df in dfdict.items():
        X = df.drop('DraftTeamSelection', axis=1)
        y = df['DraftTeamSelection']
        categorical_features = list(df.select_dtypes(include=['category', object]).columns)
        categorical_transformer = OneHotEncoder(sparse=True, handle_unknown='ignore')

        numeric_features = list(df.select_dtypes(include=['int', 'float']).columns)
        numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())
        ])

        preprocessor = ColumnTransformer(
        transformers=[
                ('num', numeric_transformer, numeric_features),
                ('cat', categorical_transformer, categorical_features)])

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
        reg = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', LinearRegression())])
        search_space = [{'regressor': [LinearRegression()],
                        'regressor__normalize': [True, False]},
                        {'regressor':[Ridge()],
                        'regressor__alpha': np.logspace(-4, 0, 50),
                        'regressor__normalize': [True, False]},
                        {'regressor': [Lasso()],
                        'regressor__alpha': np.logspace(-4, 0, 50),
                        'regressor__normalize': [True, False]},
                        {'regressor': [ElasticNet()],
                        'regressor__l1_ratio': np.linspace(0,1,30),
                        'regressor__normalize': [True, False]},]
        reg_CV = RandomizedSearchCV(reg, search_space, cv=5)
        best_model = reg_CV.fit(X_train, y_train)
        df_best_regressor = best_model.best_estimator_.get_params()['regressor']
        y_pred = best_model.predict(X_test)
        R2 = r2_score(y_test, y_pred)
        MSE = mean_squared_error(y_test, y_pred)
        RMSE = mean_squared_error(y_test, y_pred, squared=False)
        MAE = mean_absolute_error(y_test, y_pred)
        print(df_best_regressor)
        results = pd.DataFrame({"R2": R2, 'MSE':MSE, 'RMSE': RMSE, 'MAE':MAE, 'best regressor':                 df_best_regressor}, index=[key])
        dfresults = dfresults.append(results)
print(dfresults)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
           l1_ratio=0.034482758620689655, max_iter=1000, normalize=True,
           positive=False, precompute=False, random_state=None,
           selection='cyclic', tol=0.0001, warm_start=False)
Lasso(alpha=0.15264179671752318, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)
Ridge(alpha=0.05963623316594643, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=True, random_state=None, solver='auto', tol=0.001)
Ridge(alpha=0.0005428675439323859, copy_X=True, fit_intercept=True,
      max_iter=None, normalize=True, random_state=None, solver='auto',
      tol=0.001)
ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True,
           l1_ratio=0.06896551724137931, max_iter=1000, normalize=False,
           positive=False, precompute=False, random_state=None,
           selection='cyclic', tol=0.0